<a href="https://colab.research.google.com/github/SrinivasSistlaKL/Mainflow_internship_dataanalysis_task1/blob/main/task5_mainflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

# Load the dataset

df = pd.read_csv('heart.csv')

# Display basic information about the dataset
df_info = df.info()
df_head = df.head()

df_info, df_head


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


(None,
    age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
 0   52    1   0       125   212    0        1      168      0      1.0      2   
 1   53    1   0       140   203    1        0      155      1      3.1      0   
 2   70    1   0       145   174    0        1      125      1      2.6      0   
 3   61    1   0       148   203    0        1      161      0      0.0      2   
 4   62    0   0       138   294    1        1      106      0      1.9      1   
 
    ca  thal  target  
 0   2     3       0  
 1   0     3       0  
 2   0     3       0  
 3   1     3       0  
 4   3     2       0  )

In [5]:
from sklearn.preprocessing import PolynomialFeatures

# Generate polynomial features for numerical columns
numerical_columns = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(df[numerical_columns])

# Convert the polynomial features to a DataFrame and concatenate with the original data
poly_feature_names = poly.get_feature_names_out(numerical_columns)
df_poly = pd.DataFrame(poly_features, columns=poly_feature_names)

# Drop the original numerical columns and concatenate the new features
df = df.drop(columns=numerical_columns).join(df_poly)

# Display the first few rows of the updated dataset
df.head()


,sex,cp,fbs,restecg,exang,slope,ca,thal,target,age,...,trestbps^2,trestbps chol,trestbps thalach,trestbps oldpeak,chol^2,chol thalach,chol oldpeak,thalach^2,thalach oldpeak,oldpeak^2
0,1,0,0,1,0,2,2,3,0,52.0,...,15625.0,26500.0,21000.0,125.0,44944.0,35616.0,212.0,28224.0,168.0,1.00
1,1,0,1,0,1,0,0,3,0,53.0,...,19600.0,28420.0,21700.0,434.0,41209.0,31465.0,629.3,24025.0,480.5,9.61
2,1,0,0,1,1,0,0,3,0,70.0,...,21025.0,25230.0,18125.0,377.0,30276.0,21750.0,452.4,15625.0,325.0,6.76
3,1,0,0,1,0,2,1,3,0,61.0,...,21904.0,30044.0,23828.0,0.0,41209.0,32683.0,0.0,25921.0,0.0,0.00
4,0,0,1,1,0,1,3,2,0,62.0,...,19044.0,40572.0,14628.0,262.2,86436.0,31164.0,558.6,11236.0,201.4,3.61


In [6]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Standardize the features
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.drop(columns=['target']))

# Apply PCA to reduce dimensionality
pca = PCA(n_components=10)  # Keep 10 principal components
pca_features = pca.fit_transform(df_scaled)

# Convert PCA features to a DataFrame
pca_df = pd.DataFrame(pca_features, columns=[f'PC{i+1}' for i in range(10)])

# Add the target column back
pca_df['target'] = df['target'].values

# Explained variance ratio of the principal components
explained_variance_ratio = pca.explained_variance_ratio_

pca_df.head(), explained_variance_ratio


(        PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
 0 -1.312705 -0.533195  0.839273 -0.351974 -0.900146 -1.820972 -1.205787   
 1  3.347845 -2.297931  2.492154 -1.008898  0.303761 -1.228736  1.618442   
 2  4.033376 -2.472836 -0.837858 -2.812985 -0.647332 -0.263804 -1.616094   
 3 -1.346426  1.453019 -0.486182 -2.601465 -0.066226 -1.472458 -1.870753   
 4  3.452940 -0.171632 -2.397264  0.392385  0.607047  1.184210  1.364363   
 
         PC8       PC9      PC10  target  
 0  0.947009  0.332087  0.777333       0  
 1 -0.857439  0.896491 -2.258896       0  
 2 -0.820533 -0.143410 -1.901842       0  
 3  0.343033 -0.043524  0.067763       0  
 4  2.810917  1.332251  0.824368       0  ,
 array([0.2795204 , 0.19265595, 0.11170228, 0.09921061, 0.06332825,
        0.04738263, 0.03849673, 0.03280775, 0.03198376, 0.02916981]))